# Step3-Modified2 

By experiment, we find that Pytorch is much more better then Scipy while solving this problem. But unfortunately, if we want to use Pytorch instead, we need to change the format of matrix from numpy array to Pytorch Tensor. And that's why we deside to rewrite the program once again. 

Before you start, make sure you have the Pytorch Package.

In [1]:
# pip install torch

# Define forward propagation 

In [2]:
import torch
import torch.nn.functional as F

def softplus(x):
    return F.softplus(x)

def sigmoid(x):
    return torch.sigmoid(x)

In [3]:
def forward_propagation(V, b, w, w0, x):
    """
    Perform forward propagation using PyTorch.
    
    Arguments:
    V -- weight matrix for the first layer
    b -- bias matrix for the first layer
    w -- weight vector for the output layer
    w0 -- bias scalar for the output layer
    x -- input data matrix
    
    Returns:
    result -- the output of the forward propagation
    """
    V_first_column = V[:, 0].reshape(-1, 1)
    V_second_column = V[:, 1].reshape(-1, 1)
    x_first_row = x[0, :].reshape(1, -1)
    x_second_row = x[1, :].reshape(1, -1)
    b_first_column = b[:, 0].reshape(-1, 1)
    b_second_column = b[:, 1].reshape(-1, 1)
    
    parameters_ready_to_apply_softplus = torch.mm(torch.exp(V_first_column), x_first_row) + b_first_column
    parameters_ready_to_apply_sigmoid = torch.mm(torch.exp(V_second_column), x_second_row) + b_second_column
    
    after_softplus = softplus(parameters_ready_to_apply_softplus)
    after_sigmoid = sigmoid(parameters_ready_to_apply_sigmoid)

    layer1 = after_softplus * after_sigmoid

    result = torch.mm(torch.exp(w).unsqueeze(0), layer1) + torch.exp(w0)
    
    return result

Here, the input value x is a vector, looks like:

$\left(\begin{array}{cccc}K_1 & K_2 & & K_N \\ S_1 & S_2 & \cdots & S_N\end{array}\right)$

So, the output value should be a row vector: 

$\left(\hat{Y}_1, \hat{Y}_2, \hat{Y}_3 \cdots ., \hat{Y}_N\right)$

The following is the shape of other parameters: 

$V=\left(\begin{array}{cc}v_{11} & v_{12} \\ v_{21} & v_{22} \\ \vdots &\vdots \\ v_{H 1} & v_{H2}\end{array}\right) \quad b=\left(\begin{array}{cc}b_{11} & b_{12} \\ b_{21} & b_{22} \\ \vdots& \vdots \\ b_{H1} & b_{H2}\end{array}\right)$

$w=\left(w, w_2, w_3, w_4 \cdots w_H\right)$

$w_0 \in R $

# Define MSE

Here we just compute MSE: 

$ \text{MSE} = \frac{1}{m} \sum_{i=1}^{m} (AL_i - y_i)^2 $

In [4]:
def mean_squared_error(AL, Y):
    """
    Calculate Mean Squared Error (MSE) using PyTorch.
    
    Parameters:
    AL -- predicted values tensor
    Y -- true labels tensor
    
    Returns:
    mse -- mean squared error tensor
    """
    mse = torch.mean((AL - Y) ** 2)
    return mse

# Import Dataset

In [5]:
import pandas as pd
from datetime import datetime

def read_option_data(file_path, ticker, current_date):

    """
    Reads option data from an Excel file, filters it based on the given ticker and option type 'Call',
    calculates the days to expiration from the current_date, and returns matrices of strikes with days to expiration,
    and stock prices.
    
    Parameters:
    file_path (str): The path to the Excel file.
    ticker (str): The ticker symbol to filter the data.
    current_date (datetime): The current date for calculating days to expiration.
    
    Returns:
    tuple: Two matrices, one for strikes and days to expiration, and one for stock prices.
    """

    
    # Read the Excel file
    df = pd.read_excel(file_path)
    
    # Filter data for the specified ticker and for Call options
    df_filtered = df[(df['Ticker'] == ticker) & (df['Type'] == 'Call')].copy()
    
    # Convert Expiration(T) to datetime
    df_filtered['Expiration(T)'] = pd.to_datetime(df_filtered['Expiration(T)'])
    
    # Calculate the difference in days between Expiration(T) and current_date
    df_filtered['Days to Expiration'] = (df_filtered['Expiration(T)'] - current_date).dt.days
    
    # Extract K, Days to Expiration and T columns to form one matrix
    K_T_matrix = df_filtered[['Strike(K)', 'Days to Expiration']].values
    
    # Extract S column to form another matrix
    c_matrix = df_filtered[['Last Option Price (c)']].values
    
    return K_T_matrix.T, c_matrix.T

Then, we know that we should have : 

$ f \geq 0, \quad \frac{\partial f}{\partial T} \geq 0, \quad \frac{\partial f}{\partial K} \leq 0, \quad \frac{\partial^2 f}{\partial K^2} \geq 0$

So we take $ K $ as the first element, and take $ K $ as $-K$

In [6]:
def negate_first_row(matrix):
    """
    Negates the elements of the first row of the given matrix.

    Parameters:
    matrix (numpy.ndarray): The input matrix whose first row elements will be negated.

    Returns:
    numpy.ndarray: The modified matrix with the first row elements negated.
    """
    
    # Negate the first row of the matrix
    matrix[0] = -matrix[0]
    
    return matrix

In [7]:
x_apple, y_apple = read_option_data('/Users/shiyaogu/Documents/Modules in UoL/MATH 391/Attempt/options_data-2.xlsx', "AAPL",  datetime(2024, 6, 26))
x_tesla, y_tesla = read_option_data('/Users/shiyaogu/Documents/Modules in UoL/MATH 391/Attempt/options_data-2.xlsx', "TSLA",  datetime(2024, 6, 26))
x_apple_final = torch.tensor(negate_first_row(x_apple), dtype = torch.float32)
x_tesla_final = torch.tensor(negate_first_row(x_tesla), dtype = torch.float32)
y_apple_final = torch.tensor(y_apple, dtype = torch.float32)
y_tesla_final = torch.tensor(y_tesla, dtype = torch.float32)

In [8]:
import torch

def split_train_test(matrix1, matrix2, ratio):
    """
    Randomly splits two given matrices into training and testing datasets based on a given ratio.
    
    Parameters:
    matrix1 (torch.Tensor): The first input matrix.
    matrix2 (torch.Tensor): The second input matrix.
    ratio (float): The ratio of columns to include in the training data (0 < ratio < 1).
    
    Returns:
    tuple: Four matrices: train_matrix1, test_matrix1, train_matrix2, test_matrix2
    """
    # Ensure the input ratio is between 0 and 1
    if not (0 < ratio < 1):
        raise ValueError("The ratio must be a float between 0 and 1")
    
    # Calculate the number of columns for training based on the ratio
    n = int(matrix1.shape[1] * ratio)
    
    # Ensure the input matrices have enough columns
    if matrix1.shape[1] < n or matrix2.shape[1] < n:
        raise ValueError("The input matrices must have enough columns to split based on the ratio")
    
    # Randomly shuffle the indices of columns
    indices = torch.randperm(matrix1.shape[1])
    train_indices = indices[:n]
    test_indices = indices[n:]
    
    # Split the first matrix
    train_matrix1 = matrix1[:, train_indices]
    test_matrix1 = matrix1[:, test_indices]
    
    # Split the second matrix
    train_matrix2 = matrix2[:, train_indices]
    test_matrix2 = matrix2[:, test_indices]
    
    return train_matrix1, test_matrix1, train_matrix2, test_matrix2

In [9]:
train_x_apple, test_x_apple, train_y_apple, test_y_apple = split_train_test(x_apple_final, y_apple_final, 0.9)
train_x_tesla, test_x_tesla, train_y_tesla, test_y_tesla = split_train_test(x_tesla_final, y_tesla_final, 0.9)
print(train_x_apple.shape)
print(train_y_apple.shape)

torch.Size([2, 824])
torch.Size([1, 824])


# Model

In [11]:
import torch.optim as optim

# Hyperparameters
H = 100000
input_dim = 2  # Dimension of x_apple

# Initialize parameters
V = torch.randn(H, 2, requires_grad=True, dtype=torch.float32)
b = torch.randn(H, 2, requires_grad=True, dtype=torch.float32)
w = torch.randn(H, requires_grad=True, dtype=torch.float32)
w0 = torch.randn(1, requires_grad=True, dtype=torch.float32)

# Optimizer
optimizer = optim.Adam([V, b, w, w0], lr=0.01)

# Training loop
num_epochs = 50000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward propagation
    y_pred = forward_propagation(V, b, w, w0, train_x_apple)
    
    # Compute loss
    loss = mean_squared_error(y_pred, train_y_apple)
    
    # Backward propagation and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss every 1000 iterations
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Optimization finished")
optimal_V = V
optimal_b = b
optimal_w = w
optimal_w0 = w0

Epoch [100/50000], Loss: 43911.4258
Epoch [200/50000], Loss: 11511.7500
Epoch [300/50000], Loss: 5618.8203
Epoch [400/50000], Loss: 3342.1614
Epoch [500/50000], Loss: 2088.0010
Epoch [600/50000], Loss: 1502.8826
Epoch [700/50000], Loss: 1126.3840
Epoch [800/50000], Loss: 873.5991
Epoch [900/50000], Loss: 696.9634
Epoch [1000/50000], Loss: 569.3202
Epoch [1100/50000], Loss: 474.3331
Epoch [1200/50000], Loss: 401.5760
Epoch [1300/50000], Loss: 343.5269
Epoch [1400/50000], Loss: 297.5098
Epoch [1500/50000], Loss: 260.5342
Epoch [1600/50000], Loss: 230.4895
Epoch [1700/50000], Loss: 205.7788
Epoch [1800/50000], Loss: 185.1971
Epoch [1900/50000], Loss: 167.8491
Epoch [2000/50000], Loss: 152.5727
Epoch [2100/50000], Loss: 139.3475
Epoch [2200/50000], Loss: 128.3242
Epoch [2300/50000], Loss: 118.9269
Epoch [2400/50000], Loss: 110.8080
Epoch [2500/50000], Loss: 103.7824
Epoch [2600/50000], Loss: 97.6543
Epoch [2700/50000], Loss: 92.2795
Epoch [2800/50000], Loss: 87.5434
Epoch [2900/50000], Los

KeyboardInterrupt: 

# Analysis of The Model

First we try to see the relatiobship between different number of neurals: 

# Prediction Model

In [ ]:
def get_predict_function(V, b, w, w0):
    """
    Returns a prediction function using the learned parameters.

    Parameters:
    V (torch.Tensor): Learned parameter V.
    b (torch.Tensor): Learned parameter b.
    w (torch.Tensor): Learned parameter w.
    w0 (torch.Tensor): Learned parameter w0.

    Returns:
    function: A function that takes input data x and returns predictions.
    """
    def predict(x):
        """
        Predict using the learned parameters.

        Parameters:
        x (torch.Tensor): Input data of shape (n_samples, 2).

        Returns:
        torch.Tensor: The predicted values.
        """
        return forward_propagation(V,b,w,w0,x)

    return predict

In [ ]:
final_model = get_predict_function(optimal_V, optimal_b, optimal_w, optimal_w0)

In [ ]:
y_predict = final_model(test_x_apple)

In [ ]:
print(test_y_apple)

In [ ]:
print(mean_squared_error(y_predict, test_y_apple))

# Plot

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [ ]:
# 将tensor数据转换为numpy数组
test_x_apple_np = test_x_apple.detach().numpy()
test_y_apple_np = test_y_apple.detach().numpy()
y_predict_np = y_predict.detach().numpy()
# 创建三维图像
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 绘制实际值
ax.scatter(test_x_apple_np[0], test_x_apple_np[1], test_y_apple_np, c='blue', marker='o', label='Actual Values')

# 绘制预测值
ax.scatter(test_x_apple_np[0], test_x_apple_np[1], y_predict_np, c='red', marker='x', label='Predicted Values')

# 设置轴标签
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')

# 添加图例
ax.legend()

# 显示图像
plt.show()

In [ ]:
def get_predict_function_plot(V, b, w, w0):
    """
    Returns a prediction function using the learned parameters.

    Parameters:
    V (torch.Tensor): Learned parameter V.
    b (torch.Tensor): Learned parameter b.
    w (torch.Tensor): Learned parameter w.
    w0 (torch.Tensor): Learned parameter w0.

    Returns:
    function: A function that takes input data x and returns predictions.
    """
    def predict(K,T):
        """
        Predict using the learned parameters.

        Parameters:
        x (torch.Tensor): Input data of shape (n_samples, 2).

        Returns:
        torch.Tensor: The predicted values.
        """
        tensor_a = torch.tensor([K])
        tensor_b = torch.tensor([T])

         # 将它们变成列向量
        tensor_a = tensor_a.unsqueeze(1)  # 变成 1x1 矩阵
        tensor_b = tensor_b.unsqueeze(1)  # 变成 1x1 矩阵
        x = torch.cat((tensor_a, tensor_b), dim=0)
        return forward_propagation(V,b,w,w0,x).item()

    return predict

In [ ]:
final_model_plot = get_predict_function_plot(optimal_V, optimal_b, optimal_w, optimal_w0)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



# 生成 x 和 y 数据，并转换为 PyTorch 的 tensor
x = torch.linspace(0, -350, 100)
y = torch.linspace(500, 600, 100)

# 创建存储 z 数据的 tensor
z = torch.zeros((100, 100))

# 使用 for 循环计算每个 (x, y) 对应的 z 值
for i in range(100):
    for j in range(100):
        z[i, j] = final_model_plot(x[i], y[j])

# 将 PyTorch 的 tensor 转换为 NumPy 数组以便绘图
x = x.numpy()
y = y.numpy()
z = z.numpy()

# 创建网格
x, y = np.meshgrid(x, y)

# 创建图形对象
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 绘制 3D 曲面图
ax.plot_surface(y, x, z, cmap='viridis')

# 设置标签
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')

# 显示图形
plt.show()